<a href="https://colab.research.google.com/github/SeongminYun1234/DeepLearning_share/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Google Drive 강제 마운트 해제
drive.flush_and_unmount()
print("✅ 기존 마운트가 성공적으로 해제되었습니다.")

✅ 기존 마운트가 성공적으로 해제되었습니다.


In [ ]:
!rm -rf /content/drive/*
print("✅ /content/drive 디렉터리 내용이 모두 정리되었습니다.")

✅ /content/drive 디렉터리 내용이 모두 정리되었습니다.


In [ ]:
# Drive 마운트 재시도
drive.mount('/content/drive')
print("✅ Google Drive 마운트가 완료되었습니다. 이제 다음 코드를 실행하시면 됩니다.")

Mounted at /content/drive
✅ Google Drive 마운트가 완료되었습니다. 이제 다음 코드를 실행하시면 됩니다.


In [ ]:
# Colab에 Wandb를 설치 (대부분 이미 되어 있지만 확인 차원에서 실행)
!pip install wandb

# Wandb 로그인 명령어 실행
import wandb
wandb.login()

wandb: Currently logged in as: seougmin1234 (seougmin1234-study-com) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import os
import torch
import wandb
from torch import nn
import multiprocessing
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm

# PyTorch 데이터셋 및 데이터 로더 관련 Import
from torch.utils.data import DataLoader, random_split, ConcatDataset, Dataset
from torchvision import datasets
from torchvision.transforms import transforms



# --- 환경 설정 ---
# Google Drive 경로 설정 (Colab 환경에서 마운트 후 사용)
DATA_ROOT = '/content/drive/MyDrive/datasets/j_fashion_mnist'

# --- 환경 유틸리티 함수 ---
def get_num_cpu_cores():
    """사용 가능한 CPU 코어 수 반환"""
    return multiprocessing.cpu_count()

# --- Wandb Config 설정을 위한 더미 클래스 ---
class DummyWandbConfig:
    def __init__(self, cfg):
        self.__dict__ = cfg

# --- 시드 설정 함수 추가 ---
def set_seed(seed=42):
    torch.manual_seed(seed) # PyTorch CPU 난수 생성기 고정
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed) # PyTorch GPU 난수 생성기 고정
        torch.cuda.manual_seed_all(seed)
        # 결정적 알고리즘 설정 (성능 저하 가능성 있음)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    np.random.seed(seed) # NumPy 난수 생성기 고정
    os.environ['PYTHONHASHSEED'] = str(seed) # Python 해시 시드 고정 (일부 환경 변수)

# 시드 값 설정
SEED_VALUE = 42



In [ ]:
# --- 2. 기본 설정 및 하이퍼파라미터 ---
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
CHECKPOINT_PATH = '/content/drive/MyDrive/fashion_mnist_best_model.pth'

# --- Wandb Config ---
WANDB_CONFIG = {
    'batch_size': 1024,
    'epochs': 50,
    'learning_rate': 0.001,
    'weight_decay': 1e-4,
    'val_split': 0.1 # 전체 훈련 데이터의 10%를 검증에 사용
}

In [ ]:
# --- 3. 데이터 전처리 (데이터 증강) ---
class AddGaussianNoise(object):
    """텐서에 가우시안 노이즈를 추가하는 사용자 정의 Transform"""
    def __init__(self, mean=0., std=0.05):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        noise = torch.randn_like(tensor) * self.std + self.mean
        return tensor + noise

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

# --- 평균/표준편차 계산 함수 (데이터 로드 시 사용) ---
def calculate_mean_std(data_path):
    """데이터셋의 평균과 표준편차를 계산합니다."""
    # ToTensor만 적용하여 데이터셋 로드
    temp_dataset = datasets.FashionMNIST(data_path, train=True, download=True, transform=transforms.ToTensor())

    # 데이터셋의 모든 텐서를 쌓아서 평균/표준편차 계산
    imgs = torch.stack([img_t for img_t, _ in temp_dataset], dim=3)
    imgs_flat = imgs.view(1, -1)
    mean = imgs_flat.mean(dim=-1).item()
    std = imgs_flat.std(dim=-1).item()

    return mean, std

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import datasets, transforms
from torch.utils.data import Dataset as PyTorchDataset, Subset # Subset 필요

# --- 4. 데이터 로딩 (데이터셋 로드 및 분할) ---
def get_dataloaders(config):
    """훈련, 검증, 테스트 데이터 로더를 생성하고 반환합니다."""

    # 1. 평균/표준편차 계산
    f_mnist_mean, f_mnist_std = calculate_mean_std(DATA_ROOT)
    print(f"Calculated Mean: {f_mnist_mean:.4f}, Std: {f_mnist_std:.4f}")

    # 2. Transform 정의 (증강과 비증강을 분리)

    # 훈련 증강 Transform
    aug_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        # AddGaussianNoise(mean=0.0, std=0.05),
        transforms.Normalize(mean=f_mnist_mean, std=f_mnist_std)
    ])

    # 검증/테스트 Transform (비증강)
    val_test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=f_mnist_mean, std=f_mnist_std)
    ])

    # 3. 데이터셋 최초 로드 (Transform 미적용 또는 기본 적용)
    # 데이터를 인덱스로만 분리할 것이므로, 일단 val_test_transform을 적용한 상태로 로드
    full_base_dataset = datasets.FashionMNIST(
        DATA_ROOT,
        train=True,
        download=False,
        transform=val_test_transform
    )

    # 4. 훈련/검증 인덱스 분할 (시드 고정)
    set_seed(SEED_VALUE)

    # 원본 데이터셋 인덱스를 분할하여 train_base_indices와 valid_indices를 얻음
    train_size = len(full_base_dataset) - int(len(full_base_dataset) * config['val_split'])
    val_size = int(len(full_base_dataset) * config['val_split'])

    # 60k 인덱스를 훈련 인덱스와 검증 인덱스로 나눔
    train_base_indices, valid_indices = random_split(
        range(len(full_base_dataset)),
        [train_size, val_size],
        generator=torch.Generator().manual_seed(SEED_VALUE) # random_split의 generator 사용 (더 확실한 시드 고정)
    )

    # 5. 분할된 인덱스를 사용하여 Subset 생성

    # 순수 훈련 부분 (비증강) - train_subset
    train_subset = Subset(full_base_dataset, train_base_indices.indices)

    # 검증 부분 - valid_subset
    valid_subset = Subset(full_base_dataset, valid_indices.indices)

    # 증강 부분 (Subset에 Augment Transform을 적용하기 위해 데이터셋을 다시 로드/정의)
    full_aug_dataset = datasets.FashionMNIST(
        DATA_ROOT,
        train=True,
        download=False,
        transform=aug_transform
    )
    augment_subset = Subset(full_aug_dataset, train_base_indices.indices)

    # 6. 전체 훈련 데이터셋 결합
    full_train_dataset = ConcatDataset([train_subset, augment_subset]) # 원본 훈련 부분 + 증강 훈련 부분
    print(f"Total Combined Training Samples: {len(full_train_dataset)}")

    # 7. 테스트 데이터 로드
    test_dataset = datasets.FashionMNIST(DATA_ROOT, train=False, download=True, transform=val_test_transform)

    # 8. DataLoader 정의
    num_workers = get_num_cpu_cores() if os.name != 'nt' else 0

    train_loader = DataLoader(full_train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True, num_workers=num_workers)
    val_loader = DataLoader(valid_subset, batch_size=config['batch_size'], shuffle=False, pin_memory=True, num_workers=num_workers) # ⬅️ valid_subset 사용
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True, num_workers=num_workers)

    return train_loader, val_loader, test_loader

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.same_channels = (in_channels == out_channels)

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        if not self.same_channels:
            self.skip_conv = nn.Conv2d(in_channels, out_channels, 1, bias=False)
        else:
            self.skip_conv = nn.Identity()

    def forward(self, x):
        identity = self.skip_conv(x)
        out = self.conv_block(x)
        out += identity
        out = F.gelu(out)
        return out

class ResNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.feature_extractor = nn.Sequential(
            # 초기 특징 추출 (1→32)
            BasicBlock(1, 32),        # Conv 2층
            BasicBlock(32, 32),       # Conv 4층
            nn.AvgPool2d(2),             # 28x28 → 14x14

            # 중간 레벨 특징 (32→64)
            BasicBlock(32, 64),       # Conv 6층
            BasicBlock(64, 64),       # Conv 8층
            nn.AvgPool2d(2),             # 14x14 → 7x7

            # 고수준 특징 (64→128)
            BasicBlock(64, 128),      # Conv 10층
            nn.AvgPool2d(2),             # 7x7 → 4x4

            # 출력 특징 강화 (128→256)
            BasicBlock(128, 256)      # Conv 12층
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 3 * 3, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(0.5),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.5),

            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

In [ ]:
# Early Stopping 로직을 담당하는 클래스를 정의
class EarlyStoppingPyTorch:
    """val_loss가 개선되지 않으면 훈련을 중단하는 클래스"""
    def __init__(self, patience=5, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience  # 개선을 기다릴 횟수
        self.verbose = verbose    # 메시지 출력 여부
        self.counter = 0          # 개선되지 않은 횟수
        self.best_score = None    # 현재까지의 최고 점수 (최저 loss)
        self.early_stop = False   # 중단 여부 플래그
        self.val_loss_min = np.inf # 최소 손실
        self.best_val_accuracy = 0.0 # 최고 정확도 기록
        self.delta = delta        # 개선으로 간주할 최소 변화량
        self.path = path          # 최고 모델 저장 경로

    def __call__(self, val_loss, val_acc, model):
        score = -val_loss # 손실을 스코어 (최대화 목표)로 변환

        if self.best_score is None:
            self.best_score = score
            self.best_val_accuracy = val_acc # 최고 정확도 초기 기록
            self.save_checkpoint(val_loss, val_acc, model)
        elif score < self.best_score + self.delta:
            # 성능 개선 없음 (val_loss가 낮아지지 않음)
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            # 성능 개선 발생 (val_loss가 낮아짐)
            self.best_score = score
            self.best_val_accuracy = val_acc # 최고 정확도 업데이트
            self.save_checkpoint(val_loss, val_acc, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, val_acc, model):
        # 최고 성능 모델을 저장
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model (Acc: {val_acc:.4f}) ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
# --- 7. 모델 체크포인팅 및 학습 루프 함수 정의 ---
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs, checkpoint_path, early_stopper):
    """모델 훈련, 검증 및 체크포인트 저장을 수행합니다."""
    model.to(device)

    # Wandb에 모델 그래프 로깅
    wandb.watch(model, criterion, log="all", log_freq=10)

    for epoch in range(num_epochs):
        # --- 훈련 단계 ---
        model.train()
        running_loss = 0.0
        corrects_train = 0

        train_bar = tqdm(train_loader, desc=f"Train E{epoch+1}/{num_epochs}")
        for inputs, labels in train_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds_train = torch.max(outputs, 1)
            corrects_train += torch.sum(preds_train == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        train_acc = corrects_train.double() / len(train_loader.dataset)

        # --- 검증 단계 ---
        model.eval()
        val_loss = 0.0
        corrects_val = 0

        val_bar = tqdm(val_loader, desc=f"Val E{epoch+1}/{num_epochs}  ")
        with torch.no_grad():
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds_val = torch.max(outputs, 1)
                corrects_val += torch.sum(preds_val == labels.data)

        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_acc = corrects_val.double() / len(val_loader.dataset)

        early_stopper(val_epoch_loss, val_acc.item(), model)



        # 조기 종료 조건 확인
        if early_stopper.early_stop:
            print(f"\n[Early Stop!] {early_stopper.patience} Epoch 동안 Val Loss 개선 없음. 훈련을 중단합니다.")
            break

        # Wandb 로깅
        wandb.log({
            "Train/Loss": epoch_loss,
            "Val/Loss": val_epoch_loss,
            "Train/Accuracy": train_acc.item(),
            "Val/Accuracy": val_acc.item()
        }, step=epoch)

        train_bar.set_postfix({'T_Loss': f"{epoch_loss:.4f}", 'V_Acc': f"{val_acc.item():.4f}"})
        val_bar.set_postfix({'V_Loss': f"{val_epoch_loss:.4f}", 'V_Acc': f"{val_acc.item():.4f}"})

    return early_stopper.best_val_accuracy

In [ ]:
# --- 8. 모델 학습 실행 및 9. 테스트 데이터 결과 출력 ---

def evaluate_test_data(model, test_loader, device):
    """학습 완료된 모델로 테스트 데이터셋 정확도 확인"""
    model.eval()
    model.to(device)
    corrects = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

    test_acc = corrects.double() / total
    return test_acc.item()

if __name__ == "__main__":

    # 1. Wandb 초기화 및 환경 설정 (연결 실패 시 비활성화 모드)
    try:
        wandb.init(project="fashion-mnist-single-run", config=WANDB_CONFIG, settings=wandb.Settings(start_method="fork"))
        wandb_config = wandb.config
    except Exception:
        print("WandB 연결 실패. 로깅 없이 훈련을 계속합니다.")
        wandb_config = DummyWandbConfig(WANDB_CONFIG)
        wandb.init(mode="disabled")

    # 2. 데이터 로드 및 로더 생성
    train_loader, val_loader, test_loader = get_dataloaders(wandb_config)

    # 3. 모델, 손실 함수, 옵티마이저, 조기종료  정의
    model = ResNet()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=wandb_config.learning_rate, weight_decay=wandb_config.weight_decay)



    EARLY_STOPPING_PATIENCE = 50

    early_stopper = EarlyStoppingPyTorch(
        patience=EARLY_STOPPING_PATIENCE,
        verbose=True,
        path=CHECKPOINT_PATH # Early Stopping이 최고 모델을 저장할 경로 지정
    )

    # 4. 모델 훈련 및 최적 정확도 저장
    print("\n--- 모델 훈련 시작 ---")
    best_val_accuracy = train_model(
        model,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        DEVICE,
        wandb_config.epochs,
        CHECKPOINT_PATH,
        early_stopper
    )

    # 5. 테스트 모델 로드 및 평가
    print("\n--- 테스트 데이터 정확도 확인 ---")

    # 최고 성능의 모델 체크포인트 로드ㅞ
    final_model = ResNet()
    final_model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=DEVICE))

    final_model.to(DEVICE)

    test_accuracy = evaluate_test_data(final_model, test_loader, DEVICE)

    print(f"최종 모델 (최고 Val Acc: {best_val_accuracy:.4f})의 테스트 정확도: {test_accuracy:.4f}")

    # 6. Wandb 최종 요약 및 종료
    if wandb.run and not wandb.run.disabled:
        wandb.run.summary["final_test_accuracy"] = test_accuracy
        wandb.run.summary["best_validation_accuracy"] = best_val_accuracy
        wandb.run.summary["early_stopping_patience"] = EARLY_STOPPING_PATIENCE
        wandb.finish()

wandb: WARNING `start_method` is deprecated and will be removed in a future version of wandb. This setting is currently non-functional and safely ignored.


Calculated Mean: 0.2860, Std: 0.3530
Total Combined Training Samples: 108000

--- 모델 훈련 시작 ---


Train E1/50:   0%|          | 0/106 [00:00<?, ?it/s]

Val E1/50  :   0%|          | 0/6 [00:00<?, ?it/s]

Validation loss decreased (inf --> 0.268220). Saving model (Acc: 0.9022) ...


Train E2/50:   0%|          | 0/106 [00:00<?, ?it/s]

In [ ]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import multiprocessing
import torch.optim as optim


# --- 1. 환경 설정 및 상수 ---
DATA_ROOT = '/content/drive/MyDrive/datasets/j_fashion_mnist'
# 훈련 시 저장했던 체크포인트 경로
CHECKPOINT_PATH = '/content/drive/MyDrive/fashion_mnist_best_model.pth'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Fashion-MNIST 레이블 맵 (인덱스 -> 클래스 이름)
FASHION_MNIST_LABELS = {
    0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat',
    5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'
}
# (훈련 시 계산된 평균/표준편차를 사용해야 정확)
F_MNIST_MEAN = 0.2860
F_MNIST_STD = 0.3530


# --- 2. ResNet 모델 구성 요소 정의 ---

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.same_channels = (in_channels == out_channels)

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        if not self.same_channels:
            self.skip_conv = nn.Conv2d(in_channels, out_channels, 1, bias=False)
        else:
            self.skip_conv = nn.Identity()

    def forward(self, x):
        identity = self.skip_conv(x)
        out = self.conv_block(x)
        out += identity
        out = F.gelu(out)
        return out

class ResNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.feature_extractor = nn.Sequential(
            # 초기 특징 추출 (1→32)
            BasicBlock(1, 32),        # Conv 2층
            BasicBlock(32, 32),       # Conv 4층
            nn.AvgPool2d(2),             # 28x28 → 14x14

            # 중간 레벨 특징 (32→64)
            BasicBlock(32, 64),       # Conv 6층
            BasicBlock(64, 64),       # Conv 8층
            nn.AvgPool2d(2),             # 14x14 → 7x7

            # 고수준 특징 (64→128)
            BasicBlock(64, 128),      # Conv 10층
            nn.AvgPool2d(2),             # 7x7 → 4x4

            # 출력 특징 강화 (128→256)
            BasicBlock(128, 256)      # Conv 12층
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 3 * 3, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(0.5),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.5),

            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x


# --- 3. 데이터 로드 함수 ---
def get_fashion_mnist_test_data(data_path, mean, std):
    """테스트 데이터셋을 로드하고 정규화합니다."""
    test_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])
    test_dataset = datasets.FashionMNIST(data_path, train=False, download=True, transform=test_transforms)
    # 분석을 위해 전체 배치를 한 번에 로드하는 DataLoader 생성
    test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
    return test_loader


# --- 4. 분석 함수 ---
def analyze_sample_predictions(model, test_loader, device, num_samples=10, checkpoint_path=None):
    """
    10,000개의 테스트 데이터 중 임의의 10개 샘플에 대해 모델의 예측 결과를 확인하고 시각화합니다.
    """

    # 모델 로드 (체크포인트 경로가 제공되었다면)
    if checkpoint_path and os.path.exists(checkpoint_path):
        #  로드 시 장치 맵핑 적용 (GPU/CPU 불일치 방지)
        model.load_state_dict(torch.load(checkpoint_path, map_location=device))
        print(f"모델 체크포인트 로드 완료: {checkpoint_path}")

    model.eval()
    model.to(device)

    # 1. 테스트 데이터 전체 가져오기
    try:
        inputs, labels = next(iter(test_loader))
    except StopIteration:
        print("테스트 로더에 데이터가 없습니다.")
        return

    # 2. 10개의 임의 인덱스 선택
    total_indices = np.arange(len(labels))
    selected_indices = np.random.choice(total_indices, size=num_samples, replace=False)

    sample_inputs = inputs[selected_indices].to(device)
    sample_labels = labels[selected_indices].to(device)

    incorrect_predictions = []

    # 3. 예측 수행 및 시각화
    with torch.no_grad():
        outputs = model(sample_inputs)
        _, preds = torch.max(outputs, 1)

        fig, axes = plt.subplots(2, 5, figsize=(15, 6))
        axes = axes.flatten()

        for i in range(num_samples):
            pred_label = FASHION_MNIST_LABELS[preds[i].item()]
            true_label = FASHION_MNIST_LABELS[sample_labels[i].item()]
            is_correct = (preds[i] == sample_labels[i]).item()

            img = sample_inputs[i].cpu().squeeze().numpy()
            ax = axes[i]
            ax.imshow(img, cmap='gray')
            ax.set_title(f"True: {true_label}\nPred: {pred_label}",
                         color='green' if is_correct else 'red',
                         fontsize=10)
            ax.axis('off')

            # 4. 오분류 사례 기록
            if not is_correct:
                incorrect_predictions.append({
                    'index_in_batch': selected_indices[i],
                    'true_label': true_label,
                    'predicted_label': pred_label,
                    'model_output_confidences': F.softmax(outputs[i], dim=0).cpu().numpy()
                })

        plt.tight_layout()
        plt.show()

    print("\n" + "="*50)
    print(f"요약: 총 {num_samples}개 샘플 중 {len(incorrect_predictions)}개 오분류 발생")
    print("="*50)

    # 5. 오분류 해석 요청 출력
    if incorrect_predictions:
        print("오분류 사례 분석:")
        for k, item in enumerate(incorrect_predictions):
            print(f"\n[오분류 {k+1}] (원본 인덱스: {item['index_in_batch']})")
            print(f"  - 실제 레이블: {item['true_label']}")
            print(f"  - 예측 결과: {item['predicted_label']} (틀림)")
            print(f"  - 해석 작성 필요: 이 이미지를 보고 분류 결과가 틀린 이유를 분석하세요.")
    else:
        print("모든 샘플이 정확하게 분류되었습니다.")


# --- 5. 최종 실행 블록 ---
if __name__ == "__main__":

    # 1. 테스트 데이터 로드
    test_loader = get_fashion_mnist_test_data(DATA_ROOT, F_MNIST_MEAN, F_MNIST_STD)

    # 2. 모델 객체 초기화 (SimpleCNN)
    final_model = ResNet()

    # 3. 분석 실행 (저장된 모델 가중치 사용)
    print(f"\n{'='*10} 임의 샘플 10개 분류 예측 확인 및 오분류 분석 {'='*10}")

    analyze_sample_predictions(
        final_model,
        test_loader,
        DEVICE,
        num_samples=10,
        checkpoint_path=CHECKPOINT_PATH
    )